# Checking "TLD: A Vehicle Tail Light signal Dataset and  Benchmark" Dataset

In [1]:
import zipfile
import pandas as pd
import json

zip_path = "TLD-LOKI.zip"

with zipfile.ZipFile(zip_path, 'r') as z:
    all_files = z.namelist()
    
    annotation_files = [f for f in all_files if f.endswith(('.json', '.csv'))]
    
    print("Found annotation files:", annotation_files)
    
    df_list = []
    
    for ann_file in annotation_files:
        with z.open(ann_file) as f:
            if ann_file.endswith('.csv'):
                df = pd.read_csv(f)
            elif ann_file.endswith('.json'):
                data = json.load(f)
                
                # Convert JSON to DataFrame
                # Assumes JSON is a list of dicts
                if isinstance(data, list):
                    df = pd.DataFrame(data)
                else:
                    # If JSON is a dict with frames/sequences, flatten it
                    df = pd.json_normalize(data)
            else:
                continue
            
            df_list.append(df)

# Concatenate all annotations into one DataFrame
all_annotations = pd.concat(df_list, ignore_index=True)

# Inspect columns to find tail-light label column
print(all_annotations.columns)

# Example: compute counts per tail-light type
# Replace 'tail_light_state' with the actual column name
# if 'tail_light_state' in all_annotations.columns:
#     counts = all_annotations['tail_light_state'].value_counts()
#     print(counts)
# else:
#     print("Check column names above to find the tail-light label column.")


Found annotation files: ['TLD-LOKI/group_001/scenario_014/_annotations.coco.json', 'TLD-LOKI/group_001/scenario_015/_annotations.coco.json', 'TLD-LOKI/group_001/scenario_016/_annotations.coco.json', 'TLD-LOKI/group_001/scenario_017/_annotations.coco.json', 'TLD-LOKI/group_001/scenario_018/_annotations.coco.json', 'TLD-LOKI/group_001/scenario_019/_annotations.coco.json', 'TLD-LOKI/group_001/scenario_020/_annotations.coco.json', 'TLD-LOKI/group_001/scenario_056/_annotations.coco.json', 'TLD-LOKI/group_001/scenario_057/_annotations.coco.json', 'TLD-LOKI/group_001/scenario_058/_annotations.coco.json', 'TLD-LOKI/group_001/scenario_059/_annotations.coco.json', 'TLD-LOKI/group_001/scenario_060/_annotations.coco.json', 'TLD-LOKI/group_001/scenario_061/_annotations.coco.json', 'TLD-LOKI/group_001/scenario_062/_annotations.coco.json', 'TLD-LOKI/group_001/scenario_098/_annotations.coco.json', 'TLD-LOKI/group_001/scenario_100/_annotations.coco.json', 'TLD-LOKI/group_001/scenario_101/_annotations.c

In [2]:
import zipfile
import json

zip_path = "TLD-LOKI.zip"
annotation_file = "TLD-LOKI/group_001/scenario_014/_annotations.coco.json"

with zipfile.ZipFile(zip_path, 'r') as z:
    with z.open(annotation_file) as f:
        coco = json.load(f)

# Inspect top-level keys
print(coco.keys())

# Check first annotation
print(coco['annotations'][0])

# Check categories
print(coco['categories'])


dict_keys(['info', 'licenses', 'categories', 'images', 'annotations'])
{'id': 0, 'image_id': 0, 'category_id': 4, 'bbox': [802, 600, 443.71, 493.98], 'area': 219183.866, 'segmentation': [], 'iscrowd': 0}
[{'id': 0, 'name': 'cars', 'supercategory': 'none'}, {'id': 1, 'name': 'brake', 'supercategory': 'cars'}, {'id': 2, 'name': 'go', 'supercategory': 'cars'}, {'id': 3, 'name': 'right', 'supercategory': 'cars'}, {'id': 4, 'name': 'vehicle', 'supercategory': 'cars'}]


In [13]:
import zipfile
import json
import pandas as pd

zip_path = "TLD-LOKI.zip" 

all_data = []

with zipfile.ZipFile(zip_path, 'r') as zf:
    # List all JSON files inside the ZIP
    json_files = [f for f in zf.namelist() if f.endswith('_annotations.coco.json')]
    print(f"Found {len(json_files)} JSON files in ZIP.")
    
    for json_file in json_files:
        with zf.open(json_file) as f:
            coco = json.load(f)
        
        # Build category map
        category_map = {c['id']: c['name'] for c in coco['categories']}
        
        # Load annotations into DataFrame
        annotations = pd.DataFrame(coco['annotations'])
        if annotations.empty:
            continue
        
        # Map category IDs to tail-light type
        annotations['tail_light_type'] = annotations['category_id'].map(category_map)
        
        # Count annotations per type
        counts = annotations['tail_light_type'].value_counts().to_dict()
        
        # Extract scenario and group info from path
        parts = json_file.split('/')
        group = parts[1] if len(parts) > 2 else 'unknown'
        scenario = parts[2] if len(parts) > 2 else 'unknown'
        counts.update({'scenario': scenario, 'group': group})
        
        all_data.append(counts)

# Create summary DataFrame
df_summary = pd.DataFrame(all_data).fillna(0)
print(df_summary.head())
# df_summary.to_csv("tld_loki_tail_light_summary.csv", index=False)
# print("Saved summary CSV from ZIP!")

Found 642 JSON files in ZIP.
   brake  right  vehicle    go      scenario      group  left
0  279.0   33.0     11.0   2.0  scenario_014  group_001   0.0
1    0.0   15.0      1.0  78.0  scenario_015  group_001   0.0
2    0.0   12.0      0.0  97.0  scenario_016  group_001   6.0
3    0.0    0.0      4.0  40.0  scenario_017  group_001   0.0
4    0.0    0.0      1.0  47.0  scenario_018  group_001   0.0


In [14]:
total_counts = df_summary[['brake', 'right', 'left', 'go', 'vehicle']].sum()
print(total_counts)

brake      19993.0
right       2703.0
left        3083.0
go         41474.0
vehicle      920.0
dtype: float64


In [16]:
total_scenarios = len(df_summary)
print(f"Total sequences (scenarios): {total_scenarios}")

Total sequences (scenarios): 625


In [17]:
total_images = 0
with zipfile.ZipFile("TLD-LOKI.zip", 'r') as zf:
    json_files = [f for f in zf.namelist() if f.endswith('_annotations.coco.json')]
    for json_file in json_files:
        with zf.open(json_file) as f:
            coco = json.load(f)
        total_images += len(coco['images'])

print(f"Total images: {total_images}")

Total images: 40702
